In [ ]:
# !pip3 install requests
# !pip3 install python-dotenv
# !pip install gspread
# !pip install oauth2client

In [ ]:
import os
import requests
import gspread
import datetime
from oauth2client.service_account import ServiceAccountCredentials
from dotenv import load_dotenv

In [ ]:
load_dotenv('.env')

In [ ]:
# Replace with your Canvas API token
access_token = os.getenv('CANVAS_ACCESS_TOKEN')
base_url = os.getenv('CANVAS_BASE_URL')

# Headers for authentication
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Parameters to control pagination
params = {
    'per_page': 50  # Adjust this number as needed, up to the maximum allowed by the API
}

In [ ]:
conversations_response = requests.get(f'{base_url}/conversations', headers=headers, params=params)
conversations = conversations_response.json()

# List to store all messages with sender names
sender_data = []

# Get sender names from starred conversations and set status to False
for conversation in conversations:
    if conversation["starred"]:
        sender_name = conversation["participants"][0]["name"]
        sender_data.append([sender_name])  # Appending [sender_name, False] to sender_data

print("Number of starred conversations: ", len(sender_data))
print(sender_data)  # Print sender data for testing

In [ ]:
# Authenticate with Google Sheets API
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Define the folder ID where you want to save the Google Sheets file
folder_id = os.getenv('GOOGLE_DRIVE_FOLDER_ID')
# Define file name for the Google Sheets file
file_name = 'Peer Feedback List'


# Check if a file with the same name already exists
file_exists = False
for file in client.list_spreadsheet_files():
    if file['name'] == file_name:
        print("Same-name Google Sheet file already exists in the folder...")
        file_exists = True
        spreadsheet = client.open(file_name)
        break

# Get today's date and format it
today_date = datetime.datetime.now().strftime("%Y-%m-%d")

# If the file does not exist, create a new one and write sender data to it
if not file_exists:
    print('Creating a new Google Sheet file...')
    spreadsheet = client.create(file_name, folder_id=folder_id)

    # Rename the default worksheet ('Sheet1') with today's date
    worksheet = spreadsheet.sheet1
    worksheet.update_title(today_date)
    worksheet.append_row(['Sender Name'])  # Writing column headers

    # Write sender data to the worksheet
    i = 0
    for sender in sender_data:
        worksheet.append_row(sender)  # Writing sender data
        print("Added", sender[0])
        i += 1
    print("Total added:", i)

else:
    print(f'Creating a new worksheet in "{file_name}"...')
    # Add a new worksheet with today's date as the title and write sender data to it
    worksheet = spreadsheet.add_worksheet(title=today_date, rows="100", cols="20")  # Create a new worksheet
    worksheet.append_row(['Sender Name'])  # Writing column headers

    # Write sender data to the worksheet
    j = 0
    for sender in sender_data:
        # Check if the sender data already exists in sheet1
        if sender not in spreadsheet.sheet1.get_all_values():
            worksheet.append_row(sender)  # Writing sender data
            print("Added", sender[0])
            j += 1
    print("Total added:", j)

There is a bug with Google Drive. Even though I have deleted the Google Sheet file from the Google Drive UI, it still shows up in this programming interface. So, I have to delete them using the programming interface to ensure that the file is deleted.

In [ ]:
response = client.list_spreadsheet_files()

# Delete each spreadsheet file
for file in response:
    print(f"Deleting file: {file['name']} ({file['id']})")
    client.del_spreadsheet(file['id'])